In [3]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

c:\Users\itzsh\miniconda3\envs\rag\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [8]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag",dimension=1536, metric="cosine",spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': 'd4545e1cccb4cf2de6930ce3de7ca380', 'Date': 'Tue, 20 Aug 2024 20:23:05 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [2]:
import json
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. Emily Thompson',
  'subject': 'Biology',
  'stars': 4,
  'review': 'Engaging lectures and clear explanations. Challenging but fair exams.'},
 {'professor': 'Prof. Michael Chen',
  'subject': 'Computer Science',
  'stars': 5,
  'review': 'Brilliant instructor. Makes complex topics accessible and fun.'},
 {'professor': 'Dr. Sarah Johnson',
  'subject': 'Psychology',
  'stars': 3,
  'review': 'Knowledgeable, but sometimes disorganized. Office hours were helpful.'},
 {'professor': 'Prof. David Rodriguez',
  'subject': 'Mathematics',
  'stars': 4,
  'review': 'Thorough explanations, though pace can be fast. Great problem sets.'},
 {'professor': 'Dr. Lisa Patel',
  'subject': 'Chemistry',
  'stars': 5,
  'review': 'Fantastic lab instructor. Patient and encouraging with all students.'},
 {'professor': 'Prof. James Wilson',
  'subject': 'History',
  'stars': 2,
  'review': 'Dry lectures, heavy on memorization. Not very engaging.'},
 {'professor': 'Dr. Angela Martinez',
  's

In [5]:
processed_data = []
client = OpenAI()

for review in data['reviews']:
    response = client.embeddings.create(
        input=review['review'],
        model="text-embedding-3-small"
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "review": review['review'],
            "subject": review["subject"],
            "stars":  review["stars"]
        }
    })

In [6]:
processed_data[0]

{'values': [-0.01611612,
  0.0020509618,
  0.010668977,
  0.011643101,
  0.031861145,
  0.019455973,
  -0.03869989,
  0.011152726,
  0.011258753,
  0.053649712,
  0.042490363,
  -0.014499206,
  -0.053861767,
  -0.010993686,
  -0.0018124008,
  -0.006964655,
  0.0005069421,
  0.02860081,
  0.011106339,
  0.030138204,
  0.044266317,
  -0.012444931,
  0.05656546,
  0.017786046,
  -0.033743124,
  -0.073635824,
  -0.0070640557,
  -0.02347175,
  -0.013677496,
  0.028680332,
  0.08434456,
  -0.015029342,
  0.0049136933,
  -0.0137967765,
  -0.044822957,
  0.0794143,
  0.021311447,
  0.024479007,
  0.025419997,
  0.003308377,
  0.005500156,
  0.006692961,
  -0.028468277,
  -0.04800377,
  0.02088734,
  -0.020860832,
  -0.0017759539,
  -0.02699715,
  0.038673386,
  0.016500467,
  -0.014499206,
  0.03862037,
  0.09319782,
  0.011232246,
  -0.03936256,
  -0.0056857034,
  0.013637736,
  0.04643987,
  -0.017812554,
  0.00483086,
  0.06775132,
  -0.030482791,
  0.020635525,
  0.0044995253,
  -0.0145257

In [9]:
index = pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

{'upserted_count': 20}

In [10]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}